## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

In [ ]:
# Need to start spark session
from pyspark.ml.classification import LogisticRegression
from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, when, count, col
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)
spark = SparkSession.builder.appName('Basics').getOrCreate()
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
# Load training data
training = spark.read.csv("DataSets/customer_churn.csv",header=True,inferSchema=True)
training.show()
training.printSchema()
training.count()
training.head()

In [ ]:
"""
Identifying the Counts of Missing Values 
"""
training.select([count(when(col(c).isNull(), c)).alias(c) for c in 
           training.columns]).toPandas().T

In [ ]:
traininging.head()

In [ ]:
"""
my_data = training.select(['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites','Churn'])
"""


In [ ]:
"""
Preparing Dataframe for Building a Model
"""
assembler = VectorAssembler(inputCols=['Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites'],outputCol='features')

output = assembler.transform(training)
finalData = output.select('features','Churn')

"""
Splitting Data into Train & Test Split
"""
train_data, test_data = finalData.randomSplit([0.7,.3])


"""
Logistic Regression Model
"""
log_model = LogisticRegression(labelCol='Churn')
fit_model = log_model.fit(train_data)

training_sum = fit_model.summary
training_sum.predictions.show()

"""
Validate the model on the Test Data
"""
results = fit_model.evaluate(test_data)


churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                           labelCol='Churn')

auc = churn_eval.evaluate(results.predictions)
auc